In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

import os, sys
from time import time

from phm08ds.models import experiment

## Load Dataset

In [2]:
folderpath = '../../../data/interim/'
data_op_1 = pd.read_csv(folderpath + 'data_op_01.csv')
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Sensor_15,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,0.02,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,0.02,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,0.02,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,0.02,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,0.02,1,2


## Data preprocessing

### Get rid of informations there are not sensor readings

Wang (2008) reports Sensor 15 has importat information. However, there are no relevant informations of this sensor. The data seems to be corrupted like this:

Let's remove it from our database creating an object transformer.

In [3]:
from phm08ds.features.feature_selection import RemoveSensor

tf_remove_sensor_15 = RemoveSensor(sensors=[15])
data_op_1 = tf_remove_sensor_15.fit_transform(data_op_1)
data_op_1.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Operational_condition,Health_state
2,1,3,34.9986,0.8401,60.0,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1,1
11,1,12,35.0029,0.8413,60.0,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1,1
23,1,24,34.9987,0.8400,60.0,1359.28,1131.95,5.48,2222.88,183.36,2387.94,1,2
38,1,39,35.0010,0.8400,60.0,1370.11,1126.43,5.48,2222.88,183.45,2387.95,1,2
40,1,41,35.0018,0.8400,60.0,1359.77,1129.48,5.48,2222.89,182.97,2387.98,1,2


Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [4]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_1)
data_with_features.head()

,Sensor_2,Sensor_3,Sensor_4,Sensor_7,Sensor_11,Sensor_12,Health_state
2,1368.17,1122.49,5.48,2222.86,183.26,2387.95,1
11,1360.54,1130.69,5.48,2222.88,183.41,2387.95,1
23,1359.28,1131.95,5.48,2222.88,183.36,2387.94,2
38,1370.11,1126.43,5.48,2222.88,183.45,2387.95,2
40,1359.77,1129.48,5.48,2222.89,182.97,2387.98,2


We need to normalize our data. Let's use Z-score standardization.

In [5]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

array([[ 2.18753538e-01, -1.28328598e+00, -1.00000000e+00,
        -5.61707703e-01,  6.68924330e-01, -5.45027910e-01],
       [-1.13058311e+00, -1.71913583e-01, -1.00000000e+00,
        -4.90969556e-01,  1.06520396e+00, -5.45027910e-01],
       [-1.35340934e+00, -1.14172757e-03, -1.00000000e+00,
        -4.90969556e-01,  9.33110750e-01, -5.78004106e-01],
       ...,
       [ 8.83695318e-01,  9.06930839e-01, -1.00000000e+00,
        -1.12761287e+00, -1.10112469e+00, -1.40240900e+00],
       [ 1.05523615e+00,  9.55722798e-01, -1.00000000e+00,
        -1.26908916e+00, -1.60307888e+00, -1.13859943e+00],
       [ 2.31968661e+00,  1.83397806e+00, -1.00000000e+00,
        -1.69351804e+00, -2.58056864e+00, -1.83109955e+00]])

In [6]:
labels = np.array(data_with_features['Health_state'])
labels

array([1, 1, 2, ..., 4, 4, 4])

# Classification steps

## Load Experiment model

In [7]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [9]:
# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

## Put all clf in a dictionary:

In [10]:
classifiers = {'MLP': mlp_clf}

Since we are using SVM and MLP we need to extract all power from those methods. Let's perform a Random Search to parameters optimizations.

### Hyperparameter tunning

In [11]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

In [12]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))}
                  }

In [13]:
for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=10, n_iter=50, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_with_features_std, labels)
    print('Elapsed time:')
    print(time() - start)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=170 ..........................................
[CV] hidden_layer_sizes=170 ..........................................
[CV] hidden_layer_sizes=170 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_layer_sizes=833 ..........................................
[CV] hidden_la

[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed:  9.0min


[CV] . hidden_layer_sizes=173, score=0.6055312954876274, total= 4.2min
[CV] hidden_layer_sizes=610 ..........................................
[CV] . hidden_layer_sizes=603, score=0.6111111111111112, total= 9.0min
[CV] hidden_layer_sizes=610 ..........................................
[CV] .. hidden_layer_sizes=366, score=0.619533527696793, total= 9.2min
[CV] hidden_layer_sizes=738 ..........................................
[CV] . hidden_layer_sizes=568, score=0.6467153284671533, total= 5.6min
[CV] hidden_layer_sizes=738 ..........................................
[CV] . hidden_layer_sizes=322, score=0.6481751824817519, total= 4.8min
[CV] hidden_layer_sizes=738 ..........................................
[CV] . hidden_layer_sizes=322, score=0.6011644832605532, total= 5.5min
[CV] hidden_layer_sizes=738 ..........................................
[CV] . hidden_layer_sizes=577, score=0.6169590643274854, total= 6.8min
[CV] . hidden_layer_sizes=173, score=0.6501457725947521, total= 4.4min
[CV] h

[Parallel(n_jobs=100)]: Done 250 tasks      | elapsed: 20.9min


[CV] . hidden_layer_sizes=426, score=0.6096491228070176, total= 5.4min
[CV] hidden_layer_sizes=243 ..........................................
[CV] . hidden_layer_sizes=199, score=0.6055312954876274, total= 3.4min
[CV] hidden_layer_sizes=243 ..........................................
[CV] . hidden_layer_sizes=426, score=0.6443148688046647, total= 6.1min
[CV] hidden_layer_sizes=243 ..........................................
[CV] . hidden_layer_sizes=414, score=0.6122448979591837, total= 4.3min
[CV] hidden_layer_sizes=243 ..........................................
[CV] . hidden_layer_sizes=907, score=0.6038011695906432, total= 7.3min
[CV] hidden_layer_sizes=243 ..........................................
[CV] .. hidden_layer_sizes=199, score=0.618421052631579, total= 3.4min
[CV] hidden_layer_sizes=243 ..........................................
[CV] . hidden_layer_sizes=426, score=0.6364963503649635, total= 5.7min
[CV] hidden_layer_sizes=243 ..........................................
[CV] .

[Parallel(n_jobs=100)]: Done 402 out of 500 | elapsed: 31.8min remaining:  7.8min


[CV] . hidden_layer_sizes=940, score=0.6545189504373178, total=12.5min
[CV] . hidden_layer_sizes=673, score=0.6486880466472303, total= 8.8min
[CV] .. hidden_layer_sizes=825, score=0.604075691411936, total=10.3min
[CV] . hidden_layer_sizes=198, score=0.6326530612244898, total= 4.6min
[CV] . hidden_layer_sizes=825, score=0.6052631578947368, total=10.3min
[CV] .. hidden_layer_sizes=38, score=0.5997088791848617, total= 2.4min
[CV] .. hidden_layer_sizes=38, score=0.6244541484716157, total= 2.6min
[CV] . hidden_layer_sizes=673, score=0.6437956204379562, total= 9.2min
[CV] . hidden_layer_sizes=320, score=0.6111111111111112, total= 5.9min
[CV] . hidden_layer_sizes=940, score=0.6259097525473072, total=13.7min
[CV] . hidden_layer_sizes=198, score=0.6055312954876274, total= 5.3min
[CV] . hidden_layer_sizes=517, score=0.6516034985422741, total= 5.8min
[CV] . hidden_layer_sizes=795, score=0.6472303206997084, total= 8.2min
[CV] .. hidden_layer_sizes=38, score=0.6364963503649635, total= 2.2min
[CV] .

[Parallel(n_jobs=100)]: Done 500 out of 500 | elapsed: 36.7min finished


Elapsed time:
2208.553000688553


## Savel results, models and pipeline to a .pkl file 

In [14]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('remove_sensor_15', tf_remove_sensor_15),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [15]:
experiment.save_models(random_search)
experiment.save_pipeline(data_preprocessing)